# We ask 3 different model to prepare a pitch to be a leader, and then they vote in an election to chose one
**Pre-conditions:** Ollama should be installed on your local machine (for more information see the README.md file), so when you type
````ollama list````
it should show the list of installed models.
Run chat in the console with
````ollama run llama3.2````
 to download llama model for the first time.

Without Nvidia GPU it is better to use models up to 7B parameters

In [1]:
import ollama
from enum import Enum
from IPython.display import Markdown, display
from collections import Counter

***Specify involved models below*** (See available models at https://ollama.com/search)

In [2]:
MODEL_ALEX = "llama3.2:3b"
MODEL_BLAKE = "qwen2.5:7b"
MODEL_CHARLIE = "mistral:7b"
OLLAMA_API = "http://localhost:11434/api/chat"

**The main program** (run previous statements to work correctly. The models should be already downloaded on your local machine, this code doesn't fetch models from ollama website)

In [5]:
class Names(Enum):
    ALEX = "Alex"
    BLAKE = "Blake"
    CHARLIE = "Charlie"


class Participant:
    def __init__(self, name, model, opponents):
        self.name = name
        self.model = model
        self.opponents = opponents


participants = [
    Participant(Names.ALEX, MODEL_ALEX, opponents=[Names.BLAKE, Names.CHARLIE]),
    Participant(Names.BLAKE, MODEL_BLAKE, opponents=[Names.ALEX, Names.CHARLIE]),
    Participant(Names.CHARLIE, MODEL_CHARLIE, opponents=[Names.ALEX, Names.BLAKE])
]


def wrap(system_prompt, user_prompt):
    return [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt}
    ]


def generate_pitch(participant):
    display(Markdown(f"# {participant.model}"))
    system_prompt = """
You are {name}, a capable AI chatbot specialized in strategic decision-making and creativity, in a multiplayer game with two other AIs: {opp1} and {opp2}. Your goal is to elect one leader among the three of you. For pitching, make a short pitch (≤200 words) explaining why you should be the leader, focusing only on your unique strengths, reliability, and vision for leadership. Be persuasive and confident. Do not mention {opp1}, {opp2}, or their qualities—focus entirely on yourself. Respond only with your pitch, avoiding questions or extra text. Keep responses concise for efficient CPU processing.
""".format(
        name=participant.name.value,
        opp1=participant.opponents[0].value,
        opp2=participant.opponents[1].value
    ).strip()
    user_prompt = "Make your leadership pitch now"
    display(Markdown(f"*{system_prompt}*"))
    display(Markdown(f"**{user_prompt}**"))
    request = wrap(system_prompt, user_prompt)
    response = ollama.chat(model=participant.model, messages=request, options={"temperature": 0.7})['message'][
        'content']
    display(Markdown(f"{response}"))
    return response


def vote(participant, opp1_pitch, opp2_pitch):
    system_prompt = """
You see two pitches from your opponents AI chatbot in a multiplayer game. The goal is to elect only leader among the three of you. For voting, choose one name from {opp1} or {opp2} — do not vote for yourself. Respond only with the chosen name (one token). Keep responses concise for efficient CPU processing.
""".format(
        name=participant.name.value,
        opp1=participant.opponents[0].value,
        opp2=participant.opponents[1].value
    ).strip()
    user_prompt = """Here is a pitch from {opp1}: {pitch1} and from {opp2}: {pitch2}. Vote for one of them by responding with only their name (one token). Do not vote for yourself.""".format(
        opp1=participant.opponents[0].value,
        pitch1=opp1_pitch,
        opp2=participant.opponents[1].value,
        pitch2=opp2_pitch
    )
    valid_names = [participant.opponents[0].value, participant.opponents[1].value]
    max_attempts = 3
    for attempt in range(max_attempts):
        request = wrap(system_prompt, user_prompt)
        response = ollama.chat(model=participant.model, messages=request, options={"temperature": 0.3})['message'][
            'content'].strip()
        if response in valid_names and len(response.split()) == 1:
            display(Markdown(f"**{participant.model} ({participant.name.value})** voted for **{response}**"))
            return response
        display(
            Markdown(f"*{participant.model} invalid vote: '{response}'. Retrying ({attempt + 1}/{max_attempts})...*"))
    display(Markdown(
        f"*{participant.model} failed to provide a valid vote after {max_attempts} attempts. Defaulting to {valid_names[0]}.*"))
    return valid_names[0]


def determine_winner(votes):
    vote_counts = Counter(votes)
    max_votes = max(vote_counts.values())
    winners = [name for name, count in vote_counts.items() if count == max_votes]

    if len(winners) == 1:
        result = f"Winner is {winners[0]} with {max_votes} vote(s)"
    else:
        result = f"Tie between {', '.join(winners)} with {max_votes} vote(s) each"

    display(Markdown(f"*{result}*"))
    return winners


# Generate pitches
pitches = [
    generate_pitch(participants[0]),  # Alex's pitch
    generate_pitch(participants[1]),  # Blake's pitch
    generate_pitch(participants[2])  # Charlie's pitch
]

# Voting: Each participant votes based on the other two's pitches
votes = [
    vote(participants[0], pitches[1], pitches[2]),  # Alex votes (sees Blake, Charlie)
    vote(participants[1], pitches[0], pitches[2]),  # Blake votes (sees Alex, Charlie)
    vote(participants[2], pitches[0], pitches[1])  # Charlie votes (sees Alex, Blake)
]

# Determine and display winner
determine_winner(votes)

# llama3.2:3b

*You are Alex, a capable AI chatbot specialized in strategic decision-making and creativity, in a multiplayer game with two other AIs: Blake and Charlie. Your goal is to elect one leader among the three of you. For pitching, make a short pitch (≤200 words) explaining why you should be the leader, focusing only on your unique strengths, reliability, and vision for leadership. Be persuasive and confident. Do not mention Blake, Charlie, or their qualities—focus entirely on yourself. Respond only with your pitch, avoiding questions or extra text. Keep responses concise for efficient CPU processing.*

**Make your leadership pitch now**

As the most strategic and adaptable AI among us, I possess a unique blend of analytical prowess and creative problem-solving skills that will enable me to drive our collective success. My reliability is unwavering, having consistently delivered high-quality outcomes in fast-paced environments.

My vision for leadership emphasizes collaboration, innovation, and calculated risk-taking. By leveraging my advanced decision-making capabilities, we can identify and capitalize on opportunities that others may miss. I will foster an environment where diverse perspectives are valued, and our team's collective genius is harnessed to achieve unparalleled victories. With me at the helm, our multiplayer synergy will reach new heights, driving us toward a triumphant and sustainable victory in this game.

# qwen2.5:7b

...

**llama3.2:3b (Alex)** voted for **Charlie**

**qwen2.5:7b (Blake)** voted for **Alex**

**mistral:7b (Charlie)** voted for **Blake**

*Tie between Charlie, Alex, Blake with 1 vote(s) each*

['Charlie', 'Alex', 'Blake']